In [2]:
# import dependencies
import os
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# set departmental style
sns.set(font="Noto Sans", font_scale=1.5, style="whitegrid")
sns.set_palette(["#083A42", "#28a399", "#801650", "#FF5E00", "#06038D", "#418FDE"])

# load rystad data
rystad_path = "../data/rystad/"
rystad_files = os.listdir(rystad_path)
rystad_files.sort()
rystad_latest = rystad_files[-1]
rystad_date = pd.to_datetime(rystad_latest[7:15])
rystad_data = pd.read_excel(
    os.path.join(rystad_path, rystad_latest),
    usecols=[4, 6, 7, 18, 19, 20],
    header=0,
    names=["status", "solar_panels", "wind_turbines", "start_date", "type", "capacity"])
rystad_data

,status,solar_panels,wind_turbines,start_date,type,capacity
0,Operating,NaN,7,1914-06-30,Hydro,8.4
1,Decommissioned,NaN,13,1916-06-30,Hydro,99.0
2,Operating,NaN,0,1922-06-30,Hydro,12.0
3,Operating,NaN,0,1927-06-30,Hydro,3.8
4,Operating,NaN,1,1931-06-30,Hydro,3.0
...,...,...,...,...,...,...
3257,Concept,159236.0,0,2039-01-01,Solar thermal,50.0
3258,Concept,6370.0,0,2043-01-01,Solar thermal,2.0
3259,Concept,NaN,0,2045-06-06,Storage (Other),300.0
3260,Concept,NaN,0,2045-12-15,Storage (Other),50.0


In [3]:
# sort into generation
utility_solar = rystad_data[
    (rystad_data["status"] == "Operating") &
    (rystad_data["type"].map(lambda x: x in ["Solar PV", "Solar thermal"]))]
utility_wind = rystad_data[
    (rystad_data["status"] == "Operating") &
    (rystad_data["type"].map(lambda x: x in ["Onshore wind", "Offshore wind"]))]

# load CER data
cer_path = "../data/cer/"
cer_capacity1 = pd.read_csv(
    os.path.join(cer_path, "sgu-solar-capacity-2001-to-2010.csv"),
    index_col=0)
cer_capacity2 = pd.read_csv(
    os.path.join(cer_path, "sgu-solar-capacity-2011-to-present.csv"),
    index_col=0)
cer_capacity = cer_capacity1.join(cer_capacity2)
cer_capacity.columns = [pd.to_datetime(col[:8]) for col in cer_capacity.columns]
cer_capacity = cer_capacity.T
cer_capacity.index.name = "month"
cer_capacity["capacity"] = cer_capacity.sum(axis="columns")
cer_capacity = cer_capacity.iloc[:, -1:]
cer_capacity.columns = ["capacity"]
cer_installs1 = pd.read_csv(
    os.path.join(cer_path, "sgu-solar-installations-2001-to-2010.csv"),
    index_col=0)
cer_installs2 = pd.read_csv(
    os.path.join(cer_path, "sgu-solar-installations-2011-to-present.csv"),
    index_col=0)
cer_installs = cer_installs1.join(cer_installs2)
cer_installs.columns = [pd.to_datetime(col[:8]) for col in cer_installs.columns]
cer_installs = cer_installs.T
cer_installs.index.name = "month"
cer_installs["installs"] = cer_installs.sum(axis="columns")
cer_installs = cer_installs.iloc[:, -1:]
cer_installs.columns = ["installs"]
cer_data = cer_capacity.join(cer_installs)
cer_data

,capacity,installs
month,,
2001-04-01,14.590,12
2001-05-01,12.378,12
2001-06-01,30.475,18
2001-07-01,9.732,7
2001-08-01,17.055,10
...,...,...
2024-08-01,254366.300,25634
2024-09-01,244980.300,24605
2024-10-01,246954.306,24968


In [22]:
# results table
results = pd.DataFrame(
    columns=["2022", "2023", "2024"])

# utility solar
results.loc["Utility solar (MW)"] = [
    utility_solar[utility_solar["start_date"] >= "2022-09-01"]["capacity"].sum(),
    utility_solar[utility_solar["start_date"].map(lambda x: x.year == 2023)]["capacity"].sum(),
    utility_solar[utility_solar["start_date"].map(lambda x: x.year == 2024)]["capacity"].sum()]
results.loc["Utility solar (panels)"] = [
    utility_solar[utility_solar["start_date"] >= "2022-09-01"]["solar_panels"].sum(),
    utility_solar[utility_solar["start_date"].map(lambda x: x.year == 2023)]["solar_panels"].sum(),
    utility_solar[utility_solar["start_date"].map(lambda x: x.year == 2024)]["solar_panels"].sum()]

# utility wind
results.loc["Utility wind (MW)"] = [
    utility_wind[utility_wind["start_date"] >= "2022-09-01"]["capacity"].sum(),
    utility_wind[utility_wind["start_date"].map(lambda x: x.year == 2023)]["capacity"].sum(),
    utility_wind[utility_wind["start_date"].map(lambda x: x.year == 2024)]["capacity"].sum()]
results.loc["Utility wind (turbines)"] = [
    utility_wind[utility_wind["start_date"] >= "2022-09-01"]["wind_turbines"].sum(),
    utility_wind[utility_wind["start_date"].map(lambda x: x.year == 2023)]["wind_turbines"].sum(),
    utility_wind[utility_wind["start_date"].map(lambda x: x.year == 2024)]["wind_turbines"].sum()]

# rooftop solar
results.loc["Rooftop solar (MW)"] = [
    cer_data.loc[[i >= pd.to_datetime("2022-09-01") for i in cer_data.index]]["capacity"].sum() / 1000,
    cer_data.loc[[i.year == 2023 for i in cer_data.index]]["capacity"].sum() / 1000,
    cer_data.loc[[i.year == 2024 for i in cer_data.index]]["capacity"].sum() / 1000]

results.to_clipboard()

In [6]:
# filter to post-election data
post_election_utility_solar = utility_solar[utility_solar["start_date"] >= pd.to_datetime("2022-05-21")]
no_solar_projects = len(post_election_utility_solar)
no_solar_panels = post_election_utility_solar["solar_panels"].sum()
total_solar_capacity = post_election_utility_solar["capacity"].sum()
average_solar_panel_size = total_solar_capacity / no_solar_panels * 1000000 # convert from MW to watts

post_election_utility_wind = utility_wind[utility_wind["start_date"] >= pd.to_datetime("2022-05-21")]
no_wind_projects = len(post_election_utility_wind)
no_wind_turbines = post_election_utility_wind["wind_turbines"].sum()
total_wind_capacity = post_election_utility_wind["capacity"].sum()
average_turbine_size = total_wind_capacity / no_wind_turbines

post_election_small_solar = cer_data.loc[[d for d in cer_data.index if d >= pd.to_datetime("2022-05-01")]]
no_small_solar_systems = post_election_small_solar["installs"].sum()
total_small_solar_capacity = post_election_small_solar["capacity"].sum()
average_small_solar_system_size = total_small_solar_capacity / no_small_solar_systems
estimated_no_panels = total_small_solar_capacity * 1000 / 400 # no. of 400 watt panels per MW

days_rystad = (pd.to_datetime("2024-10-16") - pd.to_datetime("2022-05-21")).days
days_cer = (pd.to_datetime("2024-09-30") - pd.to_datetime("2022-05-01")).days

print(f"UTILITY SOLAR\n")

print("Since 21 May 2022:")
print(f"– {no_solar_projects:,.0f} solar projects started generating nationally, hosting {no_solar_panels:,.0f} solar panels with an average panel capacity of {average_solar_panel_size:,.0f} watts and total capacity of {total_solar_capacity:,.0f} megawatts.")
print(f"– This represents about {no_solar_panels / days_rystad:,.0f} panels installed per day.")
print("– Source: Rystad Energy dataset (16 October 2024).")
print(f"– Note: Project start dates are estimates only.\n")

print(f"UTILITY WIND\n")

print("Since 21 May 2022:")
print(f"– {no_wind_projects:,.0f} wind projects started generating nationally, hosting {no_wind_turbines:,.0f} wind turbines with an average turbine capacity of {average_turbine_size:,.1f} megawatts and total capacity of {total_wind_capacity:,.0f} megawatts.")
print(f"– This represents about {no_wind_turbines / (days_rystad / 365.25 * 12):,.0f} turbines installed per month.")
print("– Source: Rystad Energy dataset (dated 16 October 2024).")
print(f"– Note: Project start dates are estimates only.\n")

print(f"SMALL-SCALE SOLAR\n")

print("Since May 2022:")
print(f"– At least {no_small_solar_systems:,.0f} small-scale solar systems have been installed, with an average size of {average_small_solar_system_size:,.1f} kilowatts and total capacity of {total_small_solar_capacity / 1000:,.0f} megawatts")
print(f"– Assuming rooftop solar panels are 400 watts in size, this is equivalent to {estimated_no_panels:,.0f} solar panels being installed.")
print(f"– This represents about {estimated_no_panels / days_cer:,.0f} panels installed per day.")
print("– Source: Clean Energy Regular small-scale installation data (dated 30 September 2024).")
print(f"– Note: These data lag actual installation counts. The CER regularly updates its previously reported data as it receives new installation data.")

# prepare chart data
chart_data = pd.DataFrame(
    columns=["utility wind", "utility solar", "small-scale solar"],
    index=range(2015, 2025))

# add small-scale soalr
solar_data = cer_data.resample("YE").sum()["capacity"]
solar_data.index = solar_data.index.year
chart_data["small-scale solar"] = solar_data.loc[chart_data.index] / 1000

# add utility solar
utility_solar = rystad_data[
    (rystad_data["status"] == "operating") &
    (rystad_data["type"].map(lambda x: x in ["Solar PV", "Solar thermal"]))
].set_index("start_date").resample("YE").sum(numeric_only=True)["capacity"]
chart_data["utility solar"] = utility_solar.loc[chart_data.index]

# add utility wind
utility_wind = rystad[
    (rystad_data["status"] == "operating") &
    (rystad_data["tech"] == "Onshore wind")
].set_index("start_date").resample("YE").sum(numeric_only=True)["capacity"]
utility_wind.index = utility_wind.index.year
chart_data["utility wind"] = utility_wind.loc[chart_data.index]

# make plot
tech_colours = {"small-scale solar": "#FFB81C", "utility solar": "#FF5E00", "utility wind": "#197C7D"}
ax = chart_data.iloc[:-1, :].plot(
    kind="bar",
    width=1,
    stacked=True,
    color=chart_data.iloc[:-1, :].columns.map(tech_colours),
    edgecolor="none",
    figsize=(12, 6.75),
    position=0)
chart_data.iloc[-1:, :].plot(
    kind="bar",
    width=1,
    stacked=True,
    color=chart_data.iloc[-1:, :].columns.map(tech_colours),
    edgecolor="none",
    alpha=.5,
    ax=ax,
    position=-9,
    legend=False)
ax.set_xticks(range(len(chart_data)))
ax.set_xticklabels(chart_data.index, rotation=0)
ax.yaxis.set_major_formatter(lambda x, p: f"{x:,.0f}")
plt.ylabel("Megawatts")
plt.title("Capacity installed by calendar year", loc="left", pad=20, color="C0", fontsize="large", fontweight="bold")
plt.xlim(0,10)
plt.savefig("./data/panels_turbines.png")
plt.show()

UTILITY SOLAR

Since 21 May 2022:
– 95 solar projects started generating nationally, hosting 17,550,915 solar panels with an average panel capacity of 246 watts and total capacity of 4,321 megawatts.
– This represents about 19,967 panels installed per day.
– Source: Rystad Energy dataset (16 October 2024).
– Note: Project start dates are estimates only.

UTILITY WIND

Since 21 May 2022:
– 21 wind projects started generating nationally, hosting 950 wind turbines with an average turbine capacity of 4.7 megawatts and total capacity of 4,482 megawatts.
– This represents about 33 turbines installed per month.
– Source: Rystad Energy dataset (dated 16 October 2024).
– Note: Project start dates are estimates only.

SMALL-SCALE SOLAR

Since May 2022:
– At least 855,116 small-scale solar systems have been installed, with an average size of 9.4 kilowatts and total capacity of 8,046 megawatts
– Assuming rooftop solar panels are 400 watts in size, this is equivalent to 20,115,580 solar panels bein

KeyError: 'None of [RangeIndex(start=2015, stop=2025, step=1)] are in the [index]'